<a href="https://colab.research.google.com/github/yskmt2018/quantum/blob/master/integer_partition_problem_sqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integer Partition Problem with Simulated Quantum Annealing

* This notebook was created and tested on Google Colaboratory. (2020/09)

## Reference

> https://github.com/recruit-communications/pyqubo/blob/master/notebooks/japanese/integer_partition.ipynb

In [1]:
!python -V
!pip install --quiet pyqubo openjij

Python 3.6.9


In [2]:
import random
import pyqubo
from openjij import SQASampler

## Problem Settings

* 整数集合$S$が与えられる。値の和の差が0になるような二つの集合$A,B$に分割したい。

In [3]:
def random_integer(element_number):
  I = set()
  while len(I) < element_number:
    e = random.randrange(4, element_number*10, 4)
    I.add(e)
  
  return I

In [4]:
# Set of Integers
S = random_integer(100)

print('S : {}'.format(S))
print('sum:{}, len:{}'.format(sum(S), len(S)))

S : {516, 4, 520, 524, 16, 20, 536, 28, 544, 36, 556, 560, 48, 56, 576, 80, 92, 96, 616, 108, 120, 632, 640, 128, 132, 644, 144, 660, 148, 664, 156, 680, 684, 172, 688, 692, 180, 696, 184, 700, 704, 196, 204, 208, 212, 728, 732, 744, 748, 756, 760, 764, 768, 772, 784, 272, 788, 280, 284, 288, 804, 808, 816, 828, 832, 840, 340, 348, 872, 376, 904, 400, 404, 408, 416, 420, 424, 936, 940, 944, 440, 952, 444, 956, 960, 448, 968, 456, 460, 972, 976, 980, 984, 472, 476, 480, 484, 496, 500, 504}
sum:51736, len:100


## Formulation

* IPP の目的関数を定式化し、QUBO（Quadratic Unconstrained Binary Optimization）式を構築する。

* QUBO 式の構築には、OSS ライブラリ PyQUBO（[GitHub](https://github.com/recruit-communications/pyqubo), [Documentation](https://pyqubo.readthedocs.io/en/latest/)）を用いる。

### QUBO: Objective Function

* 目的関数：二つの集合の値の和の差

In [5]:
def build_objective():
  H = 0.0
  for s in S:
    H += s * pyqubo.Spin('{}'.format(s))

  return H**2

### Hamiltonian

* 目的関数からハミルトニアン H を定義する。

In [6]:
H = build_objective()
model = H.compile()
qubo, constant = model.to_qubo()

## Execute Quantum Annealing

* Sampler と呼ばれるモジュールを生成し、構築した QUBO 式を渡すことで、量子アニーリングを実行する。

* 結果は、Response オブジェクトとして返却される。

### OpenJij

* OpenJij（[GitHub](https://github.com/OpenJij/OpenJij), [Documentation](https://openjij.github.io/OpenJij_Documentation/build/html/), [Tutorial](https://openjij.github.io/OpenJijTutorial/build/html/ja/index.html)）

* OSS として、量子アニーリングをシミュレートする SQA（Simulated Quantum Annealing）の Python 用 API を提供している。

In [7]:
sampler = SQASampler(num_reads=10)

In [8]:
%%time
response = sampler.sample_qubo(qubo)

CPU times: user 1.35 s, sys: 853 ms, total: 2.2 s
Wall time: 1.18 s


### Take Solutions

* Response オブジェクトに格納されている量子アニーリングの結果（解）を取り出す。

In [9]:
def extract_samples(response):
  solutions = []
  energies = []
  
  for record in response.record:
    sol, num_occ = record[0], record[2]
    solution, _, energy = model.decode_solution(dict(zip(response.variables, sol)), vartype='BINARY')
    solutions += [solution] * num_occ
    energies += [energy] * num_occ
  
  return solutions, energies

In [10]:
solutions, energies = extract_samples(response)
best_solution = solutions[energies.index(min(energies))]

### Print Integer Partition

* 二つの集合$A,B$及び値の和の差を表示する。

In [11]:
A = set()
B = set()

for (k, v) in best_solution.items():
  if v == 0:
    A.add(int(k))
  else:
    B.add(int(k))

In [12]:
print('A : {}'.format(A))
print('sum:{}, len:{}'.format(sum(A), len(A)))
print('=================')
print('B : {}'.format(B))
print('sum:{}, len:{}'.format(sum(B), len(B)))
print('=================')
print('diff:{}'.format(abs(sum(A)-sum(B))))

A : {128, 640, 132, 4, 644, 520, 16, 784, 148, 20, 408, 664, 156, 288, 416, 36, 804, 936, 172, 556, 940, 688, 180, 692, 440, 696, 952, 444, 700, 828, 576, 196, 840, 972, 208, 976, 212, 340, 728, 984, 120, 348, 476, 480, 96, 484, 632, 748, 496, 760, 376, 764}
sum:25844, len:52
B : {768, 516, 772, 904, 524, 144, 272, 400, 404, 660, 788, 280, 536, 284, 28, 544, 420, 424, 680, 808, 684, 48, 560, 816, 944, 184, 56, 956, 448, 704, 832, 960, 456, 968, 204, 460, 80, 980, 472, 732, 92, 616, 744, 872, 108, 500, 756, 504}
sum:25892, len:48
diff:48
